In [1]:
""" 제 1고지 미분 자동 계산 """

' 제 1고지 미분 자동 계산 '

In [2]:
""" STEP7. 역전파 자동화 """


# Variable이라는 상자 생성
class Variable:
    def __init__(self, data): # 생성자
        self.data = data
        self.grad = None # 미분값 저장하기 위한 변수

        # 변수(Variable)가 어떤 연산에 의해 생성되었는지를 나타내는 속성
        self.creator = None # 연산을 나타내는 객체

    # 해당 변수가 어떤 함수에 의해 만들어졌는지를 저장
    def set_creator(self, func):
        self.creator = func

In [3]:
# Variable 인스턴스를 변수로 다룰 수 있는 함수를 Function클래스로 구현
class Function:
    def __call__(self, input):
        x = input.data # 데이터를 꺼냄
        y = self.forward(x) # 구체적인 계산 == forward 메서드에서 함
        output = Variable(y)  # Variable 형태로 되돌림
        output.set_creator(self) # 출력 변수에 창조자 설정
        self.input = input # 입력 저장
        self.output = output # 출력 저장
        return output
    
    # 순전파
    def forward(self, x):
        raise NotImplementedError()
    
    # 역전파
    def backward(self, gy):
        raise NotImplementedError()

In [4]:
# y = x²
class Square(Function):
    # 순전파
    def forward(self, x):
        y = x ** 2 # y = x²
        return y
    
    # 역전파
    def backward(self, gy): # gy = 출력쪽에 전해지는 미분값을 전달하는 역할
        x = self.input.data
        gx = 2 * x * gy #  y' = 2x
        return gx

In [5]:
import numpy as np

# y = eˣ
class Exp(Function):
    # 순전파
    def forward(self, x):
        y = np.exp(x) # 주어진 입력값에 대한 지수 함수를 계산하여 반환
        return y
    
    # 역전파
    def backward(self, gy):
        x = self.input.data
        """ 지수 함수의 도함수는 자기 자신을 유지하므로 
            입력값의 지수 함수 값에 gy를 곱함 """
        gx = np.exp(x) * gy
        return gx

In [6]:
# 합성함수 y = (eˣ²)²
A = Square()
B = Exp()
C = Square()

# 순전파
x = Variable(np.array(0.5))
a = A(x) # x ** 2
b = B(a) # e ** a
y = C(b) # b ** 2

# 계산 그래프의 노드들을 거꾸로 거슬러 올라감
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [7]:
# 역전파
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다.
b = C.input # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다.

In [8]:
B = b.creator # 1. 함수를 가져온다.
a = B.input # 2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드를 호술한다.

In [9]:
A = a.creator # 1. 함수를 가져온다.
x = A.input # 함수의 입력을 가져온다.
x.grad = A.backward(a.grad) # 함수의 backward 메서드를 호출한다.
print(x.grad)

3.297442541400256
